## Setting Up Gym ENV

In [1]:
!pip install tensorflow==2.3.1 gym keras-rl2 gym[atari]

/lib.linux-x86_64-3.8/atari_py/ale_interface/src/games/supported
    copying atari_py/ale_interface/src/games/supported/Venture.hpp -> build/lib.linux-x86_64-3.8/atari_py/ale_interface/src/games/supported
    copying atari_py/ale_interface/src/games/supported/VideoPinball.cpp -> build/lib.linux-x86_64-3.8/atari_py/ale_interface/src/games/supported
    copying atari_py/ale_interface/src/games/supported/VideoPinball.hpp -> build/lib.linux-x86_64-3.8/atari_py/ale_interface/src/games/supported
    copying atari_py/ale_interface/src/games/supported/WizardOfWor.cpp -> build/lib.linux-x86_64-3.8/atari_py/ale_interface/src/games/supported
    copying atari_py/ale_interface/src/games/supported/WizardOfWor.hpp -> build/lib.linux-x86_64-3.8/atari_py/ale_interface/src/games/supported
    copying atari_py/ale_interface/src/games/supported/YarsRevenge.cpp -> build/lib.linux-x86_64-3.8/atari_py/ale_interface/src/games/supported
    copying atari_py/ale_interface/src/games/supported/YarsRevenge.hpp ->

In [35]:
import gym
import random
import numpy as np

In [36]:
env = gym.make('SpaceInvaders-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [37]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [38]:
episodes = 5
for episode in range(episodes):
    state = env.reset()
    done = False
    score = 0
    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info, = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:0 Score:45.0
Episode:1 Score:60.0
Episode:2 Score:100.0
Episode:3 Score:240.0
Episode:4 Score:135.0


## Create a Deep Learning Model with Keras 

In [39]:
import numpy as np 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [40]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(
        Convolution2D(32,(8,8), 
        strides=(4,4), 
        activation = 'relu',
        input_shape=(3, height, width, channels)))
    model.add(
        Convolution2D(64,(4,4), 
        strides=(2,2), 
        activation = 'relu'))
    model.add(
        Convolution2D(64,(3,3), 
        activation = 'relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [23]:
#del model

In [41]:
model = build_model(height, width, channels, actions)

In [42]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_15 (Dense)             (None, 512)               34603520  
_________________________________________________________________
dense_16 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_17 (Dense)             (None, 6)                

## Build agent with Keras-RL

In [20]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [26]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, enable_dueling_network=True, dueling_type='avg', nb_actions=actions, nb_steps_warmup=1000)
    return dqn

In [27]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))
dqn.fit(env, nb_steps=10000, visualize=False, verbose=1)

Training for 10000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 5666s 567ms/step - reward: 0.2315
done, took 5665.802 seconds


In [28]:
dqn.save_weights('10k_fast/dqn_10k_fast.h5f')

In [57]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 335.000, steps: 680
Episode 2: reward: 485.000, steps: 1692
Episode 3: reward: 575.000, steps: 1209
Episode 4: reward: 110.000, steps: 616
Episode 5: reward: 345.000, steps: 1243
Episode 6: reward: 175.000, steps: 613
Episode 7: reward: 200.000, steps: 674
Episode 8: reward: 90.000, steps: 530
Episode 9: reward: 260.000, steps: 983
Episode 10: reward: 655.000, steps: 1632
323.0


## Load Model From Memory 1m steps

In [59]:
del model, dqn

In [61]:
model = build_model(height, width, channels, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))
dqn.load_weights('1m/dqn_weights.h5f')

In [62]:
scores = dqn.test(env, nb_episodes=5, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 5 episodes ...
Episode 1: reward: 170.000, steps: 633
Episode 2: reward: 300.000, steps: 816
Episode 3: reward: 295.000, steps: 1170
Episode 4: reward: 335.000, steps: 925
Episode 5: reward: 85.000, steps: 517
237.0
